In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Load libraries

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import random

from sklearn.metrics import confusion_matrix

import tensorflow as tf
tf.random.set_seed(0)
tf.keras.backend.clear_session

# Load data

In [ ]:
path = '/kaggle/input/challenges-in-representation-learning-facial-expression-recognition-challenge/'
tdat = pd.read_csv(path+'icml_face_data.csv')
tdat.sample(5)

In [ ]:
emotions = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

# Preprocess data

In [ ]:
tdat.info()

In [ ]:
dat = tdat.copy()
dat.drop_duplicates(inplace=True)
dat.info()

In [ ]:
dat[' Usage'].unique()

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3,figsize=(20,5))
sns.countplot(data = dat[dat[' Usage']=='Training'], x='emotion', ax=ax1).set_title('Training')
ax1.set_xticklabels(emotions.values())
sns.countplot(data = dat[dat[' Usage']=='PublicTest'], x='emotion', ax=ax2).set_title('Testing')
ax2.set_xticklabels(emotions.values())
sns.countplot(data = dat[dat[' Usage']=='PrivateTest'], x='emotion', ax=ax3).set_title('Validation')
ax3.set_xticklabels(emotions.values())

# Helper functions

In [ ]:
def prepare_data(data):
    """ Prepare data for modeling 
        input: data frame with labels und pixel data
        output: image and label array """
    
    image_array = np.zeros(shape=(len(data), 48, 48))
    image_label = np.array(list(map(int, data['emotion'])))
    
    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, ' pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48))
        image_array[i] = image
        
    return image_array, image_label

  
def sample_plot(x,y=None):
    #x, y are numpy arrays
    n = 20
    samples = random.sample(range(x.shape[0]),n)
    
    fig, axs = plt.subplots(2,10, figsize=(25,5), sharex=True, sharey=True)
    ax = axs.ravel()
    for i in range(n):
        ax[i].imshow(x[samples[i],:,:], cmap=plt.get_cmap('gray'))
        ax[i].set_xticks([])
        ax[i].set_yticks([])
        if y is not None:
            ax[i].set_title(emotions[y[samples[i]]])
              

# Prepare data for CNN, accounting for class imbalance

In [ ]:
from imblearn.under_sampling import RandomUnderSampler, TomekLinks
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.pipeline import Pipeline

from collections import Counter

In [ ]:
train_image_array1, train_image_label1 = prepare_data(dat[dat[' Usage']=='Training'])
val_image_array, val_image_label = prepare_data(dat[dat[' Usage']=='PrivateTest'])
test_image_array, test_image_label = prepare_data(dat[dat[' Usage']=='PublicTest'])

In [ ]:
# pp = Pipeline([('rus',RandomUnderSampler(random_state=0, replacement=False))])
pp = Pipeline([('tk',TomekLinks()),('ros',RandomOverSampler(random_state=0))])
train_image_array, train_image_label = pp.fit_resample(train_image_array1.reshape(train_image_array1.shape[0],48*48), train_image_label1)
print(Counter(train_image_label))
train_image_array = train_image_array.reshape(train_image_array.shape[0], 48, 48)

In [ ]:
train_images = train_image_array.reshape((train_image_array.shape[0], 48, 48, 1))
train_images = train_images.astype('float32')
val_images = val_image_array.reshape((val_image_array.shape[0], 48, 48, 1))
val_images = val_images.astype('float32')
test_images = test_image_array.reshape((test_image_array.shape[0], 48, 48, 1))
test_images = test_images.astype('float32')

train_labels = tf.keras.utils.to_categorical(train_image_label)
val_labels = tf.keras.utils.to_categorical(val_image_label)
test_labels = tf.keras.utils.to_categorical(test_image_label)

# Sample and visualize images from train/valid/test data sets

In [ ]:
sample_plot(train_image_array, train_image_label)

In [ ]:
sample_plot(val_image_array, val_image_label)

In [ ]:
sample_plot(test_image_array, test_image_label)

In [ ]:
wt = dat[dat[' Usage']=="Training"].groupby('emotion').agg('count')
# class_weights = 
wt['fraction'] = wt[' pixels']/np.sum(wt[' pixels'])
class_weights = dict(zip(range(7), wt.fraction))

# CNN

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(scale=1./255, input_shape=(48,48,1)),
    tf.keras.layers.experimental.preprocessing.RandomContrast(factor = 0.2),
    tf.keras.layers.experimental.preprocessing.RandomFlip(mode='horizontal'),
    
    tf.keras.layers.Conv2D(16,3,activation='relu',padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(16,5,activation='relu',padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(16,3,activation='relu',padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.MaxPooling2D(2),
    
    tf.keras.layers.Conv2D(16,3,activation='relu',padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(16,3,activation='relu',padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(emotions), activation='softmax'),    
])

In [ ]:
from sklearn.metrics import f1_score

def eager_f1score(y_true, y_pred):
    return f1_score(y_true=np.argmax(np.array(y_true), axis=1), y_pred=np.argmax(np.array(y_pred),axis=1), average='macro')


# eager_f1score(test_image_label, test_pred)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='categorical_crossentropy', 
              metrics=['accuracy',eager_f1score], run_eagerly=True)

earlystop = tf.keras.callbacks.EarlyStopping(patience=10, min_delta=1e-4, restore_best_weights=True)
lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.5, min_lr=1e-7)

In [ ]:
hist = model.fit(train_images, train_labels,
                    validation_data=(val_images, val_labels),
                    epochs=50,
#                     class_weight=class_weights,
                    batch_size=128,
                    callbacks=[earlystop, lr])

In [ ]:
for key in hist.history.keys():
    plt.plot(hist.history[key], label=key)
plt.legend()

In [ ]:
model.evaluate(test_images, test_labels)
test_pred = model.predict(test_images)
confusion_matrix(y_true=test_image_label, y_pred=np.argmax(test_pred,axis=1))